### 初期設定

In [27]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
from utils.data_loarder import load_csv_to_db, get_pivot_from_db
import utils.config as cfg
import utils.data_visualize as dv
import utils.data_pivot_from_DB as dp
print("🔁 autoreload 有効化完了")
import os
import pandas as pd
import plotly.express as px
import yaml

# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🔁 autoreload 有効化完了


{'project_name': 'stock_screening',
 'data_path': {'raw': './data/raw'},
 'files': {'raw': {'資産詳細': '資産詳細.csv',
   '収支詳細': '収支詳細.csv',
   '資産目標値': '資産目標値.csv'}},
 'database_path': './database',
 'database': {'finance': 'finance.db'}}

In [28]:
csv_files = {}
for file in settings["files"]["raw"]:
    csv_files[settings["files"]["raw"][file]] = os.path.join(settings["data_path"]["raw"],"",settings["files"]["raw"][file]) 
#print(csv_files)
DB_PATH_FINANCE = os.path.join(settings["database_path"],"",settings["database"]["finance"])

load_csv_to_db(
    csv_files=csv_files,
    db_path=DB_PATH_FINANCE
)

✅ ./data/raw\資産詳細.csv → テーブル '資産詳細' に登録しました。
✅ ./data/raw\収支詳細.csv → テーブル '収支詳細' に登録しました。
✅ ./data/raw\資産目標値.csv → テーブル '資産目標値' に登録しました。
🎉 すべてのCSVを1つのDBに統合しました！


### 動作確認

#### _________ダッシュボード

In [ ]:
graphs = []   
# 1. 総資産推移
pivot = get_pivot_from_db(DB_PATH_FINANCE, "資産詳細", "date", ["資産タイプ"],["資産額"],"sum").sum(axis=1)

end_date = pivot.index.max() + pd.DateOffset(months=1)
where_clause = f"date <= DATE('{end_date}')"
pivot_target = get_pivot_from_db(DB_PATH_FINANCE, "資産目標値", "date", ["資産タイプ"],["資産額"],"sum",where_clause=where_clause).sum(axis=1)

df = pd.concat([pivot, pivot_target], axis=1, keys=["実績", "目標"])

fig = px.line(df, x=df.index, y=df.columns, title="総資産推移", labels={"date": "日付", "values":"資産額","variable":""}, template="plotly_dark")
fig.update_xaxes(tickformat="%y/%m/%d")
fig.update_layout(autosize=True, margin=dict(l=10,r=10,t=30,b=30))
fig.show()

# 2. トータルリターン推移
pivot = get_pivot_from_db(DB_PATH_FINANCE, "資産詳細", "date", ["資産タイプ"],["トータルリターン"],"sum").sum(axis=1)

end_date = pivot.index.max() + pd.DateOffset(months=1)
where_clause = f"date <= DATE('{end_date}')"
pivot_target = get_pivot_from_db(DB_PATH_FINANCE, "資産目標値", "date", ["資産タイプ"],["トータルリターン"],"sum",where_clause=where_clause).sum(axis=1).cumsum()

df = pd.concat([pivot, pivot_target], axis=1, keys=["実績", "目標"])

fig = px.line(df, x=df.index, y=df.columns, title="トータルリターン推移", labels={"date": "日付", "values":"リターン額","variable":""}, template="plotly_dark")
fig.update_xaxes(tickformat="%y/%m/%d")
fig.update_layout(autosize=True, margin=dict(l=10,r=10,t=30,b=30))
fig.show()

# 3. 任意グラフ（後で追加可能）
# 4. 一般収支推移


# 5. 特別収支推移
# 6. 任意グラフ（後で追加可能）

TypeError: 'numpy.dtypes.ObjectDType' object is not callable

### SQLite関連

In [ ]:
conn = sqlite3.connect(DB_PATH_FINANCE)
cur = conn.cursor()

# .csv付きのテーブルを削除
for table in ["資産詳細.csv", "収支詳細.csv", "資産目標値.csv"]:
    cur.execute(f'DROP TABLE IF EXISTS "{table}"')
    print(f"削除: {table}")

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect(DB_PATH_FINANCE)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
conn.close()

print(tables)